# This notebook demonstrates to transform the output of CLUMPY to a template we can use for the HAWC analysis

In [1]:
# import the necessary packages
import astropy.io.fits as fits
from astropy.coordinates import SkyCoord
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt # may not be necessary
import matplotlib.colors as colors
import healpy as hp
from scipy.interpolate import interp2d
import pickle
import os

from matplotlib import rcParams
rcParams['figure.figsize'] = [8.0, 6.0]
rcParams['xtick.labelsize'] = 14
rcParams['ytick.labelsize'] = 14
rcParams['axes.labelsize'] = 14
rcParams['legend.fontsize'] = 11

In [4]:
def generate_cartesian_file(hdus, lon, lat, label, index=1):
    content = hdus[index].header
    theta_0 = content['THETA_0']
    psi_0   = content['PSI_0']
    size_x  = content['SIZE_X']
    size_y  = content['SIZE_Y']
    dangle  = content['ALPHAINT']

    content = hdus[index].data
    pixels  = content['PIXEL']
    Jtotal  = content['Jtot']
    Jsmooth = content['Jsmooth']
    Jsub    = content['Jsub']
    Jcross  = content['Jcrossp']
    NSIDE    = hdus[2].header['NSIDE']
    ordering = hdus[2].header['ORDERING']
    if ordering == "NESTED":
        nested = True
    else:
        nested = False

    nPix = hp.nside2npix(NSIDE)
    num_used_pixels = len(pixels)
    theta_rad, phi_rad = hp.pixelfunc.pix2ang(NSIDE, pixels, nest=nested)
    # change the angles according to our conventions
    theta, phi = -np.degrees(theta_rad-np.pi/2.-lon), np.degrees(np.pi*2.-phi_rad+lat)
    phi[np.where(phi>360)] -= 360.
    # Process the data for 3ML use
    nxPix = 280
    nyPix = 280
    refX  = 140.5
    refY  = 140.5
    delX  = -dangle
    delY  = dangle
    x = np.zeros(nPix)
    x[pixels] = Jtotal/np.max(Jtotal)
    dmROI = np.zeros([nxPix, nyPix])
    for i in range(nxPix):
        for j in range(nyPix):
            ra_roi = (i-np.int(refX))*delX                                                                          
            dec_roi = (j-np.int(refY))*delY
            hpix = hp.pixelfunc.ang2pix(NSIDE,np.radians(-dec_roi+90.),np.radians(360.-ra_roi), nest=nested)
            #print(hpix)
            dmROI[i,j] = x[hpix]
    # convert from galactic coordinates to fk5
    dmROI = np.multiply(dmROI, (np.degrees(1)**2/delX**2))
    coords = SkyCoord(lon*u.degree, lat*u.degree, frame='galactic', unit='degree')
    RA  = coords.fk5.ra.degree
    DEC = coords.fk5.dec.degree
    # write the output to the new file
    new_hdu = fits.PrimaryHDU(dmROI)
    new_hdu.header['CTYPE1'] = 'RA'
    new_hdu.header['CTYPE2'] = 'DEC'
    new_hdu.header['CUNIT1'] = 'deg'
    new_hdu.header['CUNIT2'] = 'deg'
    new_hdu.header['CRPIX1'] = refX
    new_hdu.header['CRPIX2'] = refY
    new_hdu.header['CRVAL1'] = RA
    new_hdu.header['CRVAL2'] = DEC
    new_hdu.header['CDELT1'] = delX
    new_hdu.header['CDELT2'] = delY
    #new_hdu.header['DMAX']   = np.log10(Jtotal.max())
    hdulist = fits.HDUList([new_hdu])
    hdulist.writeto('{}_Jfactor_template.fits'.format(label))
    print("For {}: Jmax={}".format(label, np.log10(Jtotal.max())))

In [5]:
# M87
lon_M87 = 283.77
lat_M87 = 74.49
# M49
lon_M49 = 286.92
lat_M49 = 70.20
# Virgo Cluster
lon_Virgo = 283.77
lat_Virgo = 74.49

M87_coords = SkyCoord(lon_M87*u.degree, lat_M87*u.degree, frame='galactic', unit='degree')
M87_lat = M87_coords.galactic.l.radian
M87_lon = M87_coords.galactic.b.radian

M49_coords = SkyCoord(lon_M49*u.degree, lat_M49*u.degree, frame='galactic', unit='degree')
M49_lat = M49_coords.galactic.l.radian
M49_lon = M49_coords.galactic.b.radian

"""
M87_filename    = "../output/annihil_M872D_FOVdiameter12.0deg_rse5_alphaint0.11deg_nside1024.fits"
M49_filename    = "../output/annihil_M492D_FOVdiameter12.0deg_rse5_alphaint0.11deg_nside1024.fits"
VirgoC_filename = "../output/annihil_VirgoC2D_FOVdiameter12.0deg_rse5_alphaint0.11deg_nside1024.fits"
"""
M87_filename    = "../output/annihil_M87_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
M49_filename    = "../output/annihil_M49_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
VirgoC_filename = "../output/annihil_VirgoC_GAO2D_FOVdiameter14.0deg_rse1_alphaint0.06deg_nside2048.fits"
"""
M87_filename    = "../output/annihil_M87_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.11deg_nside1024.fits"
M49_filename    = "../output/annihil_M49_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.11deg_nside1024.fits"
VirgoC_filename = "../output/annihil_VirgoC_NFW2D_FOVdiameter14.0deg_rse1_alphaint0.11deg_nside1024.fits"
"""
M87_hdus = fits.open(M87_filename)
M49_hdus = fits.open(M49_filename)
Virgo_hdus = fits.open(VirgoC_filename)

for i in range(1, len(M87_hdus)):
    print("header {}: {}".format(i, M87_hdus[i].header['EXTNAME']))
    
for i in range(1, len(M49_hdus)):
    print("header {}: {}".format(i, M49_hdus[i].header['EXTNAME']))

"""
generate_cartesian_file(M87_hdus, lon_M87, lat_M87, "M87_nfw")
generate_cartesian_file(M49_hdus, lon_M49, lat_M49, "M49_nfw")
generate_cartesian_file(Virgo_hdus, lon_Virgo, lat_Virgo, "VirgoC_nfw")
"""
generate_cartesian_file(M87_hdus, lon_M87, lat_M87, "M87_gao")
generate_cartesian_file(M49_hdus, lon_M49, lat_M49, "M49_gao")
generate_cartesian_file(Virgo_hdus, lon_Virgo, lat_Virgo, "VirgoC_gao")

header 1: JFACTOR
header 2: JFACTOR_PER_SR
header 3: INTEGRATED_FLUXES
header 1: JFACTOR
header 2: JFACTOR_PER_SR
header 3: INTEGRATED_FLUXES
For M87_gao: Jmax=17.6983528137
For M49_gao: Jmax=17.2624816895
For VirgoC_gao: Jmax=19.0107460022


## Read the data from the fits generated with CLUMPY

In [ ]:
content = hdus[1].header
theta_0 = content['THETA_0']
psi_0   = content['PSI_0']
size_x  = content['SIZE_X']
size_y  = content['SIZE_Y']
dangle  = content['ALPHAINT']

content = hdus[1].data
pixels  = content['PIXEL']
Jtotal  = content['Jtot']
Jsmooth = content['Jsmooth']
Jsub    = content['Jsub']
Jcross  = content['Jcrossp']
NSIDE    = hdus[2].header['NSIDE']
ordering = hdus[2].header['ORDERING']
if ordering == "NESTED":
    nested = True
else:
    nested = False

nPix = hp.nside2npix(NSIDE)
num_used_pixels = len(pixels)
theta_rad, phi_rad = hp.pixelfunc.pix2ang(NSIDE, pixels, nest=nested)
# change the angles according to our conventions
theta, phi = -np.degrees(theta_rad-np.pi/2.-virgo_lon), np.degrees(np.pi*2.-phi_rad+virgo_lat)
phi[np.where(phi>360)] -= 360.

print("max log10(J)/sr is {}".format(np.log10(Jtotal.max())))

In [ ]:
plt.clf()
Jtot = np.ones(nPix)
Jtot[pixels] = Jtotal
hp.cartview(Jtot, latra=[-7., 7.], lonra=[-7., 7.], nest=nested, 
            min=1e21, norm='log', title="J factor per sr", unit="J/sr")
plt.savefig("../original_template.pdf", bbox_inches='tight')
plt.savefig("../original_template.png", bbox_inches='tight')
plt.savefig("../original_template.eps", bbox_inches='tight')
plt.show()

## This section explains how to transform the CLUMPY generated file to a file that can be used in AERIE

In [ ]:
if os.path.isfile("interpolator1.pkl"):
    # pickles exist. so only read it and save time
    print("pickles exist... I will read the interpolators from the pickles...")
    interpolator1 = pickle.load(open("interpolator1.pkl", "rb"))
    interpolator2 = pickle.load(open("interpolator2.pkl", "rb"))
else:
    print("pickles are missing... I will run the analysis for interpolators...")
    interpolator1 = interp2d(theta[:25000], phi[:25000], np.log10(Jtotal[:25000]), bounds_error=True)
    interpolator2 = interp2d(theta[25000:], phi[25000:], np.log10(Jtotal[25000:]), bounds_error=True)
    with open('interpolator1.pkl', 'wb') as f:
        pickle.dump(interpolator1, f)
    with open('interpolator2.pkl', 'wb') as f:
        pickle.dump(interpolator2, f)

In [ ]:
pixAngle = hp.pix2ang(NSIDE, range(nPix), nest=nested)
lon, lat = -np.degrees(pixAngle[0]-np.pi/2.), np.degrees(np.pi*2.-pixAngle[1])

In [ ]:
pix1 = []
j_factors1 = []
for i in range(nPix):
    try:
        j_value  = interpolator1(lon[i], lat[i])[0]
        if j_value > 26:
            continue
        pix1.append(i)
        j_factors1.append(j_value)
    except:
        continue
    
pix2 = []
j_factors2= []
for i in range(nPix):
    try:
        j_value  = interpolator2(lon[i], lat[i])[0]
        if j_value > 26:
            continue
        pix2.append(i)
        j_factors2.append(j_value)
    except:
        continue

Jtot = np.zeros(nPix)
Jtot[pix1] = j_factors1
Jtot[pix2] = j_factors2

In [ ]:
Jtot = np.zeros(nPix)
Jtot[pix1] = j_factors1
for i in range(len(pix2)):
    if Jtot[pix2[i]] < 20:
        Jtot[pix2[i]] = j_factors2[i]
plt.clf()
Jtot[np.where(Jtot<=21)] = 0
Jtot[np.where(Jtot>=26)] = 0
Jtot = 10**Jtot
hp.cartview(Jtot, latra=[67., 82.], lonra=[70., 90.], nest=nested, min=1e21, max=1e26,
           title="J factor per sr", unit="log(J)/sr", norm='log')
plt.savefig("../translated_template.pdf", bbox_inches='tight')
plt.savefig("../translated_template.png", bbox_inches='tight')
plt.savefig("../translated_template.eps", bbox_inches='tight')
plt.show()
Jtot = Jtot/np.max(Jtot)

In [ ]:
new_hdu = fits.PrimaryHDU(Jtot)
new_hdu.header['CTYPE1'] = 'RA'
new_hdu.header['CTYPE2'] = 'DEC'
new_hdu.header['CUNIT1'] = 'deg'
new_hdu.header['CUNIT2'] = 'deg'
new_hdu.header['CRPIX1'] = 70.5
new_hdu.header['CRPIX2'] = 70.5
new_hdu.header['CRVAL1'] = RA
new_hdu.header['CRVAL2'] = DEC
new_hdu.header['CDELT1'] = -dangle
new_hdu.header['CDELT2'] = dangle
hdulist = fits.HDUList([new_hdu])
hdulist.writeto('VirgoCluster_Jfactor_template.fits')